In [1]:
data_path = "./data/"
sp_data_path = data_path + "SP-train.npy"
wp_data_path = data_path + "WP-train.npy"

In [2]:
import numpy as np
import torch
import transformers
import datasets
from datasets import load_dataset, load_metric
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import AutoTokenizer
import wandb

if torch.cuda.is_available():
    device = torch.device("cuda:0")

# do this once
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

/home/qianqi/.conda/envs/nlp243/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/qianqi/.conda/envs/nlp243/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
print(torch.cuda.is_available())

False


In [4]:
sp_train = np.load(sp_data_path)
wp_train = np.load(wp_data_path)

In [5]:
print(len(sp_train))
print(len(wp_train))

507
396


In [6]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [7]:
random.seed(0)
train_data_sp, val_data_sp = train_test_split(sp_train, test_size=0.1, random_state = 17)
train_data_wp, val_data_wp = train_test_split(wp_train, test_size=0.1, random_state = 17)

In [7]:
def combine_sp_wp(sp, wp):
    combined_p = [x for x in sp]
    for x in wp:
        combined_p.append(x)
    return combined_p

In [8]:
train_data = combine_sp_wp(sp_train, wp_train)

In [9]:
train_data = combine_sp_wp(train_data_sp, train_data_wp)
val_data = combine_sp_wp(val_data_sp, val_data_wp)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [10]:
print(len(train_data))
print(train_data[0])
print(train_data[-1])

903
{'id': 'SP-0', 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'answer': 'Each daughter shares the same brother.', 'distractor1': 'Some daughters get married and have their own family.', 'distractor2': 'Some brothers were not loved by family and moved away.', 'distractor(unsure)': 'None of above.', 'label': 1, 'choice_list': ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.'], 'choice_order': [1, 0, 2, 3]}
{'id': 'WP-163_CR', 'question': "What kind of ice doesn't contain water?", 'answer': 'Dry ice.', 'distractor1': 'Flaked ice.', 'distractor2': 'Glacier ice.', 'distractor(unsure)': 'None of above.', 'label': 1, 'choice_list': ['Flaked ice.', 'Dry ice.', 'Glacier ice.', 'None of above.'], 'choice_order': [1, 0, 2, 3]}


In [11]:
def preprocess_function(data):
    # Repeat each quesiton four times to go with the four possibilities of second sentences.
    questions = [[i["question"]] * 4 for i in data]
    # Grab all choices possible for each context.
    choices = [i["choice_list"] for i in data]
    # Flatten everything
    questions = sum(questions, [])
    choices = sum(choices, [])
    # Tokenize
    tokenized_qa = tokenizer(questions, choices, truncation=True, padding=True)
    # Un-flatten
    return {k: [v[i : i+4] for i in range(0, len(v), 4)] for k, v in tokenized_qa.items()}

In [12]:
examples = train_data[:5]
# 'input_ids', 'token_type_ids', 'attention_mask', (#data, 4, #feature)
features = preprocess_function(examples) 
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] a woman shoots her husband. then she holds him underwater for over 5 minutes. finally, she hangs him. but 5 minutes later, they both go out and enjoy a wonderful dinner together. how can this be? [SEP] the woman gets arrested for murder after dinner. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] a woman shoots her husband. then she holds him underwater for over 5 minutes. finally, she hangs him. but 5 minutes later, they both go out and enjoy a wonderful dinner together. how can this be? [SEP] the woman gets a new partner. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] a woman shoots her husband. then she holds him underwater for over 5 minutes. finally, she hangs him. but 5 minutes later, they both go out and enjoy a wonderful dinner together. how can this be? [SEP] the woman was a photographer. she shot a picture of her husband, developed it, and 

In [13]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="wandb"
)

In [18]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [20]:
accepted_keys = ["input_ids", "attention_mask"]
train_features = preprocess_function(train_data) 
for i in range(len(train_data)):
    for k in accepted_keys:
        train_data[i][k] = train_features[k][i]

# val_features = preprocess_function(val_data) 
# for i in range(len(val_data)):
#     for k in accepted_keys:
#         val_data[i][k] = val_features[k][i]

In [21]:
accepted_keys = ["input_ids", "attention_mask", "label"]
# 'input_ids', 'attention_mask', (#data, 4, #feature)
train_features = [{k: v for k, v in train_data[i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(train_features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [22]:
[tokenizer.decode(batch["input_ids"][-1][i].tolist()) for i in range(4)]

['[CLS] five apples are in a basket. how do you divide them among five girls so that each girl gets an apple, but one apple remains in the basket? [SEP] two girls decide to share one apple. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[C

In [23]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    # eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [26]:
print(model.device)

cpu


In [25]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: qyan79 (nlp243). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [99]:
trainer.save_model("./bert-base-uncased-finetuned/")

In [101]:
test_args = TrainingArguments(
    output_dir = "sample-test",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = batch_size,   
    dataloader_drop_last = False    
)
# init trainer
trainer = Trainer(
              model = model, 
              args = test_args, 
              compute_metrics = compute_metrics)
test_results = trainer.predict(val_data)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
